In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [12]:
import torch
import timm
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from PIL import Image
import pandas as pd
import numpy as np
import os

# Define class labels
class_columns = ['Angioectasia', 'Bleeding', 'Erosion', 'Erythema', 'Foreign Body', 
                 'Lymphangiectasia', 'Normal', 'Polyp', 'Ulcer', 'Worms']

# Define transformations
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Define dataset paths
train_data_dir = '/kaggle/input/capsule-vision-2024-challenge/Dataset/training'
val_data_dir = '/kaggle/input/capsule-vision-2024-challenge/Dataset/validation'
test_data_dir = '/kaggle/input/testing/Testing set/Images'

# Load datasets
train_dataset = datasets.ImageFolder(root=train_data_dir, transform=train_transforms)
val_dataset = datasets.ImageFolder(root=val_data_dir, transform=test_transforms)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Define the Swin Transformer model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
swin_model = timm.create_model('swin_tiny_patch4_window7_224', pretrained=True, num_classes=len(class_columns))
swin_model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = Adam(swin_model.parameters(), lr=0.0001)

# Training function
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    total_loss, correct, total = 0.0, 0, 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        correct += (outputs.argmax(1) == labels).sum().item()
        total += labels.size(0)
    return total_loss / len(train_loader), 100 * correct / total

# Validation function
def validate(model, val_loader, criterion, device):
    model.eval()
    total_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            total_loss += loss.item()
            correct += (outputs.argmax(1) == labels).sum().item()
            total += labels.size(0)
    return total_loss / len(val_loader), 100 * correct / total

# Train and validate the model
epochs = 10
best_val_accuracy = 0

for epoch in range(epochs):
    train_loss, train_accuracy = train(swin_model, train_loader, criterion, optimizer, device)
    val_loss, val_accuracy = validate(swin_model, val_loader, criterion, device)
    
    print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}% - Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}%")
    
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(swin_model.state_dict(), 'best_swin_model.pth')
        print("Model saved!")

# Load the best model for testing
swin_model.load_state_dict(torch.load('best_swin_model.pth'))
swin_model.eval()

# Custom Dataset for Test Data
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.image_paths = [os.path.join(image_dir, img) for img in os.listdir(image_dir) if img.endswith(('.png', '.jpg', '.jpeg'))]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, img_path

# Test DataLoader
test_dataset = TestDataset(test_data_dir, transform=test_transforms)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Function to save predictions to CSV
def save_predictions_to_csv(model, dataloader, output_csv, device):
    model.eval()
    predictions = []
    with torch.no_grad():
        for images, image_paths in dataloader:
            images = images.to(device)
            outputs = model(images)
            probs = torch.nn.functional.softmax(outputs, dim=1).cpu().numpy()
            predicted_classes = np.argmax(probs, axis=1)
            
            for i, img_path in enumerate(image_paths):
                pred_row = [img_path] + list(probs[i]) + [class_columns[predicted_classes[i]]]
                predictions.append(pred_row)

    # Create DataFrame and save to CSV
    columns = ['image_path'] + class_columns + ['predicted_class']
    df = pd.DataFrame(predictions, columns=columns)
    df.to_csv(output_csv, index=False)
    print(f"Predictions saved to {output_csv}")

# Run the prediction function
save_predictions_to_csv(swin_model, test_loader, 'test_predictions.csv', device)


Epoch 1/10 - Train Loss: 0.3869, Train Acc: 87.80% - Val Loss: 0.2726, Val Acc: 90.64%
Model saved!
Epoch 2/10 - Train Loss: 0.2121, Train Acc: 92.88% - Val Loss: 0.2373, Val Acc: 92.29%
Model saved!
Epoch 3/10 - Train Loss: 0.1652, Train Acc: 94.51% - Val Loss: 0.2379, Val Acc: 92.50%
Model saved!
Epoch 4/10 - Train Loss: 0.1357, Train Acc: 95.47% - Val Loss: 0.2246, Val Acc: 93.08%
Model saved!
Epoch 5/10 - Train Loss: 0.1132, Train Acc: 96.13% - Val Loss: 0.2088, Val Acc: 93.48%
Model saved!
Epoch 6/10 - Train Loss: 0.1029, Train Acc: 96.50% - Val Loss: 0.2093, Val Acc: 93.39%
Epoch 7/10 - Train Loss: 0.0859, Train Acc: 97.06% - Val Loss: 0.2389, Val Acc: 93.19%
Epoch 8/10 - Train Loss: 0.0778, Train Acc: 97.38% - Val Loss: 0.2455, Val Acc: 92.93%
Epoch 9/10 - Train Loss: 0.0690, Train Acc: 97.65% - Val Loss: 0.2501, Val Acc: 93.45%
Epoch 10/10 - Train Loss: 0.0718, Train Acc: 97.53% - Val Loss: 0.2615, Val Acc: 93.15%


/tmp/ipykernel_30/1205500640.py:101: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  swin_model.load_state_dict(torch.load('best_swin_model.pth'))


Predictions saved to test_predictions.csv
